In [1]:
import pandas as pd
import numpy as np

In [2]:
class Match:      
    def __init__(self, score):  
        self.score = score
        self.split_score = self.get_split_score(score)
        self.num_sets = self.get_num_sets(self.split_score)
        self.sets = []
        for set_num in range(self.num_sets):
            self.sets.append(
            {'set_num': TennisSet(self.split_score[set_num], set_num+1).set_num, 
             'set_score': TennisSet(self.split_score[set_num], set_num+1).set_score,
             'set_tiebreaker': TennisSet(self.split_score[set_num], set_num+1).tiebreaker,
             'match_winner_games': TennisSet(self.split_score[set_num], set_num+1).w_games,
             'match_loser_games': TennisSet(self.split_score[set_num], set_num+1).l_games,
             'match_winner_tb': TennisSet(self.split_score[set_num], set_num+1).w_tb_pts,
             'match_loser_tb': TennisSet(self.split_score[set_num], set_num+1).l_tb_pts,
             'set_winner_games': TennisSet(self.split_score[set_num], set_num+1).set_w_games,
             'set_loser_games': TennisSet(self.split_score[set_num], set_num+1).set_l_games,
             'set_winner_tb': TennisSet(self.split_score[set_num], set_num+1).tb_winner_pts,
             'set_loser_tb': TennisSet(self.split_score[set_num], set_num+1).tb_loser_pts
            })
    
    def get_split_score(self, score):
#         if score == "W/O":
#             return 'walkover'
#         elif 'RET' in score:
#             return 'retirement'
#         else:
        split_score = score.split(' ')
        return split_score
        
    def get_num_sets(self, split_score):
#         if 'walkover' in split_score or 'retirement' in split_score:
#             num_sets = 0
#         else:
        num_sets = len(split_score)
        return num_sets
        
class TennisSet():
    def __init__(self, set_score, set_num):
        self.set_num = set_num
        self.set_score = set_score
        self.w_games = self.get_winner_games(self.set_score)
        self.l_games_split = self.get_loser_games_split(set_score)
        self.l_games = self.get_loser_games(self.l_games_split)
        self.set_w_games = self.get_set_winner_games(self.w_games, self.l_games)
        self.set_l_games = self.get_set_loser_games(self.l_games, self.w_games)
        self.tiebreaker = self.get_tb_bool(self.set_score)
        self.tb_loser_pts = self.get_tb_loser_pts(self.l_games_split)
        self.tb_winner_pts = self.get_tb_winner_pts(self.tb_loser_pts)
        self.l_tb_pts = self.get_match_loser_tbs(self.l_games, self.tb_loser_pts, self.tb_winner_pts)
        self.w_tb_pts = self.get_match_winner_tbs(self.w_games, self.tb_loser_pts, self.tb_winner_pts)

    def get_winner_games(self, set_score):
        return int(set_score.split('-')[0])
    
    def get_loser_games_split(self, set_score):
        l_games_split = set_score.split('-')[1]
        return l_games_split

    def get_loser_games(self, l_games_split):
        if "(" in l_games_split:
            temp = l_games_split.split("(")
            l_games = temp[0]
        else:
            l_games = l_games_split
        return int(l_games) 
    
    def get_set_winner_games(self, w_games, l_games):
        if w_games > l_games:
            return w_games
        elif l_games > w_games:
            return l_games
        else:
            return 'e'
    
    def get_set_loser_games(self, l_games, w_games):
        if l_games > w_games:
            return w_games
        elif w_games > l_games:
            return l_games
        else:
            return 'e'
    
    def get_tb_bool(self, l_games_split):
        if "(" in l_games_split:
            return True
        else:
            return False
        
    def get_tb_loser_pts(self, l_games_split):
        if "(" in l_games_split:
            tb_loser_pts  = l_games_split.split("(")[1]
            tb_loser_pts = int(tb_loser_pts.replace(")",""))
            return tb_loser_pts
        else:
            return np.nan
        
    def get_tb_winner_pts(self, tb_loser_pts):
        if tb_loser_pts is not np.nan:
            if tb_loser_pts +2 >= 7:
                tb_winner_pts =tb_loser_pts +2
            else:
                tb_winner_pts = 7
        else:
            tb_winner_pts = np.nan
        return tb_winner_pts
        
    def get_match_loser_tbs(self, l_games, tb_loser_pts, tb_winner_pts):
        if tb_loser_pts is not np.nan:
            if l_games == 7:
                l_tb_pts = tb_winner_pts
            else:
                l_tb_pts = tb_loser_pts
            return l_tb_pts
        else:
            return np.nan

    def get_match_winner_tbs(self, w_games, tb_loser_pts, tb_winner_pts):
        if tb_loser_pts is not np.nan:
            if w_games == 7:
                w_tb_pts = tb_winner_pts
            else:
                w_tb_pts = tb_loser_pts
            return w_tb_pts
        else:
            return np.nan

Example Match object containing Set objects

In [6]:
test = '6-7(4) 6-0 7-6(10)'

test_match = Match(test)

In [7]:
test_match.sets

[{'set_num': 1,
  'set_score': '6-7(4)',
  'set_tiebreaker': True,
  'match_winner_games': 6,
  'match_loser_games': 7,
  'match_winner_tb': 4,
  'match_loser_tb': 7,
  'set_winner_games': 7,
  'set_loser_games': 6,
  'set_winner_tb': 7,
  'set_loser_tb': 4},
 {'set_num': 2,
  'set_score': '6-0',
  'set_tiebreaker': False,
  'match_winner_games': 6,
  'match_loser_games': 0,
  'match_winner_tb': nan,
  'match_loser_tb': nan,
  'set_winner_games': 6,
  'set_loser_games': 0,
  'set_winner_tb': nan,
  'set_loser_tb': nan},
 {'set_num': 3,
  'set_score': '7-6(10)',
  'set_tiebreaker': True,
  'match_winner_games': 7,
  'match_loser_games': 6,
  'match_winner_tb': 12,
  'match_loser_tb': 10,
  'set_winner_games': 7,
  'set_loser_games': 6,
  'set_winner_tb': 12,
  'set_loser_tb': 10}]

Let's look at some attributes from the first set

In [5]:
test_match.sets[0]['set_score']

'6-7(4)'

This set went to a set tiebreaker 

The first player to win 7 points in the tiebreaker wins the set, as long as there's a 2 point differential between the players

For example, if you're up 7-5, you win the set; if you're up 7-6, you have to play on

In [8]:
test_match.sets[0]['set_tiebreaker']

True

The match winner's score is reported first for all sets regardless of if they won the set

In this case the match winner lost the set (and tiebreaker)

In [7]:
test_match.sets[0]['match_winner_games']

6

In [8]:
test_match.sets[0]['match_winner_tb']

4

There may be times you want to know info based on who won the set instead of the match

In [9]:
test_match.sets[0]['set_winner_games']

7

In [10]:
test_match.sets[0]['set_winner_tb']

7